In [1]:
import os

import numpy as np
import obspy
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from scipy import signal, fft
import pprint as pp

import utils

#plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 20, 5
plt.rcParams['lines.linewidth'] = 0.5

## load inSight data to solar
solar = utils.Solar('VF', 'B').get_solar()
#pp.pprint(solar)

trace = solar['S0986c']['VEL']
traceE = trace[0].copy()
traceN = trace[1].copy()
traceZ = trace[2].copy()

data = traceZ.data
npts = traceZ.stats.npts
delta = traceZ.stats.delta
fs = traceZ.stats.sampling_rate
xticks = np.arange(0, npts*delta, delta)
t = delta*np.linspace(0, npts-1, npts)
yf = {}
xf = fft.fftfreq(npts, delta)
yf['E'] = fft.fft(traceE)
yf['N'] = fft.fft(traceN)
yf['Z'] = fft.fft(traceZ)

In [2]:
import scipy.io as io
io.savemat('data.mat', {"yf_E": yf['E'],
                        "yf_N": yf['N'],
                        "yf_Z": yf['Z'],
                        "xf": xf})

In [35]:
import numpy as np
from scipy.signal import detrend
from scipy.signal import cheb1ord
from scipy.signal import cheby1

def raydec1station(vert=None, north=None, east=None, time=None, fmin=None, fmax=None, fsteps=None, cycles=None, dfpar=None, nwind=None):
    # RAYDEC1STATION(VERT, NORTH, EAST, TIME, FMIN, FMAX, FSTEPS, CYCLES, DFPAR, NWIND)
    #    calculates the ellipticity of Rayleigh waves for the
    #    input data VERT, NORTH, EAST and TIME for a single station
    #    for FSTEPS frequencies (on a logarithmic scale) between
    #    FMIN and FMAX, using CYCLES periods for the stacked signal
    #    and DFPAR as the relative bandwidth for the filtering.
    #    The signal is cut into NWIN different time windows and
    #    RayDec is applied to each of them.

    #    VERT, NORTH, EAST and TIME have to be data matrices
    #    (N x 1 or 1 x N) of equal sizes

    #    suggested values: CYCLES = 10
    #                      DFPAR = 0.1
    #                      NWIND such that the single time windows are about 10 minutes long

    #    Code written by Manuel Hobiger,
    #    Laboratoire de Géophysique Interne et Tectonophysique (LGIT), Grenoble, France, 2008-10
    #    Last change: 18/06/2021

    v1 = vert
    n1 = north
    e1 = east
    t1 = time
#    if v1.shape[1] > v1.shape[0]:
#        v1 = transpose(v1)
#        n1 = transpose(n1)
#        e1 = transpose(e1)
#        t1 = transpose(t1)

    # setting up
    K0 = v1.shape[0]
    K = int(np.floor(K0 / nwind))
    tau = t1[2] - t1[1]
    DTmax = 30
    fnyq = 1 / (2 * tau)
    fstart = max(fmin, 1./DTmax)
    fend = min(fmax, fnyq)
    flist = np.zeros((fsteps, 1))
    constlog = (fend / fstart) ** (1 / (fsteps - 1))
    fl = fstart * constlog ** (np.cumsum(np.ones((fsteps, nwind)), axis=0) - 1)
    el = np.zeros((fsteps, nwind))
    # loop over the time windows
    for ind1 in np.arange(1, nwind+1).reshape(-1):
        vert = detrend(v1[np.arange((ind1 - 1) * K + 1, ind1 * K+1)])
        north = detrend(n1[np.arange((ind1 - 1) * K + 1, ind1 * K+1)])
        east = detrend(e1[np.arange((ind1 - 1) * K + 1, ind1 * K+1)])
        time = t1[np.arange((ind1 - 1) * K + 1, ind1 * K+1)]
        horizontalamp = np.zeros((fsteps, 1))
        verticalamp = np.zeros((fsteps, 1))
        horizontallist = np.zeros((fsteps, 1))
        verticallist = np.zeros((fsteps, 1))
        Tmax = np.amax(time)
        thetas = np.zeros((fsteps, int(np.ceil(Tmax * fend))))
        corr = np.zeros((fsteps, int(np.ceil(Tmax * fend))))
        ampl = np.zeros((fsteps, int(np.ceil(Tmax * fend))))
        dvmax = np.zeros((fsteps, 1))
        for findex in np.arange(1, fsteps+1, 1).reshape(-1):
            f = fl[findex, ind1]
            df = dfpar * f
            fmin = max(fstart, f - df / 2)
            fmax = min(fnyq, f + df / 2)
            flist[findex] = f
            DT = cycles / f
            wl = np.round(DT / tau)
            na, wn = cheb1ord(np.array([fmin + (fmax - fmin) / 10, fmax - (fmax - fmin) / 10]) /
                              fnyq, np.array([fmin - (fmax - fmin) / 10, fmax + (fmax - fmin) / 10]) / fnyq, 1, 5)
            ch1, ch2 = cheby1(na, 0.5, np.array(wn), btype='bandpass')
            taper1 = np.arange(
                0, 1+1 / np.round(time.shape[0] / 100), 1 / np.round(time.shape[0] / 100))
            print(taper1.shape)
            taper2 = np.ones((1, time.shape[0] - taper1.shape[0] * 2))
            taper3 = np.fliplr(taper1)
            taper = np.array([taper1, taper2, taper3]).T
            # filtering the signals
            norths = np.filter(ch1, ch2, np.multiply(taper, north))
            easts = np.filter(ch1, ch2, np.multiply(taper, east))
            verts = np.filter(ch1, ch2, np.multiply(taper, vert))
            derive = (np.sign(verts(np.arange(2, K+1))) -
                      np.sign(verts(np.arange(1, (K - 1)+1)))) / 2
            vertsum = np.zeros((wl, 1))
            horsum = np.zeros((wl, 1))
            dvindex = 0
            for index in np.arange(np.ceil(1 / (4 * f * tau)) + 1, len(derive) - wl+1, 1).reshape(-1):
                if derive(index) == 1:
                    dvindex = dvindex + 1
                    vsig = verts(np.arange(index, (index + wl - 1)+1))
                    esig = easts(np.arange(index - int(np.floor(1 / (4 * f * tau))),
                                 (index - int(np.floor(1 / (4 * f * tau))) + wl - 1)+1))
                    nsig = norths(np.arange(index - int(np.floor(1 / (4 * f * tau))),
                                  (index - int(np.floor(1 / (4 * f * tau))) + wl - 1)+1))
                    integral1 = sum(np.multiply(vsig, esig))
                    integral2 = sum(np.multiply(vsig, nsig))
                    theta = np.arctan(integral1 / integral2)
                    if integral2 < 0:
                        theta = theta + np.pi
                    theta = np.mod(theta + np.pi, 2 * np.pi)
                    hsig = np.sin(theta) * esig + np.cos(theta) * nsig
                    correlation = sum(np.multiply(
                        vsig, hsig)) / np.sqrt(sum(np.multiply(vsig, vsig)) * sum(np.multiply(hsig, hsig)))
                    if correlation >= - 1:
                        vertsum = vertsum + correlation ** 2 * vsig
                        horsum = horsum + correlation ** 2 * hsig
                    thetas[findex, dvindex] = theta
                    correlationlist[index] = correlation
                    thetalist[index] = theta
                    corr[findex, dvindex] = correlation
                    dvmax[findex] = dvindex
                    ampl[findex, dvindex] = sum(vsig ** 2 + hsig ** 2)
            klimit = np.round(DT / tau)
            verticalamp[findex] = np.sqrt(
                sum(vertsum(np.arange(1, klimit+1)) ** 2))
            horizontalamp[findex] = np.sqrt(
                sum(horsum(np.arange(1, klimit+1)) ** 2))
        ellist = horizontalamp / verticalamp
        fl[:, ind1] = flist
        el[:, ind1] = ellist

    V = fl
    W = el
    return V, W


In [36]:
raydec1station(yf['E'], yf['N'], yf['Z'], t, 1., 50, 100, 10, 1., 10)

(145,)


ValueError: Input must be >= 2-d.

In [39]:
time = np.ones((10000))
taper1 = np.arange(
                0, 1+1 / np.round(time.shape[0] / 100), 1 / np.round(time.shape[0] / 100))